In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erfc

# Parameters
num_symbols = 100000  # Number of symbols (increase for better accuracy at low BER)
snr_db = np.arange(0, 21, 2)  # SNR range in dB
ber = np.zeros(len(snr_db))  # To store BER values

# OTFS Parameters
M = 8  # Number of delay bins
N = 8  # Number of Doppler bins
num_bits = num_symbols  # Total number of bits (adjusted for correct reshaping)

# OTFS Modulation
def otfs_modulation(data):
    # Reshape data into delay-Doppler domain
    data_matrix = data[:M * N].reshape(M, N)  # Ensure correct reshaping
    # Perform ISFFT (Inverse Symplectic Finite Fourier Transform)
    time_freq_matrix = np.fft.ifft(np.fft.fft(data_matrix, axis=1), axis=0)
    return time_freq_matrix

# OTFS Demodulation
def otfs_demodulation(received_signal):
    # Perform SFFT (Symplectic Finite Fourier Transform)
    data_matrix = np.fft.fft(np.fft.ifft(received_signal, axis=0), axis=1)
    # Reshape back to bitstream
    data = data_matrix.flatten()
    return np.real(data) > 0  # BPSK decision

# Delay-Doppler Channel (simplified)
def delay_doppler_channel(signal):
    # Add a simple delay-Doppler effect (for simulation purposes)
    h = np.random.randn() * np.exp(1j * np.random.randn())  # Random complex gain
    return h * signal

# Simulation
for i, snr in enumerate(snr_db):
    # Generate random data (BPSK: 0s and 1s)
    data = np.random.randint(0, 2, M * N)  # Adjusted to match reshaping constraints
    
    # OTFS Modulation
    modulated_signal = otfs_modulation(data)
    
    # Pass through delay-Doppler channel
    channel_output = delay_doppler_channel(modulated_signal)
    
    # Add AWGN noise
    snr_linear = 10 ** (snr / 10)
    noise_power = 1 / snr_linear
    noise = np.sqrt(noise_power / 2) * (np.random.randn(*channel_output.shape) + 1j * np.random.randn(*channel_output.shape))
    received_signal = channel_output + noise
    
    # OTFS Demodulation
    demodulated_data = otfs_demodulation(received_signal)
    
    # Calculate BER
    errors = np.sum(data != demodulated_data[:M * N])
    ber[i] = errors / (M * N)
    
    # Stop simulation if BER is below 1e-6 (to save time)
    if ber[i] < 1e-6:
        break

# Theoretical BER for BPSK (for comparison)
theoretical_ber = 0.5 * erfc(np.sqrt(10 ** (snr_db / 10)))

# Plotting
plt.figure(figsize=(10, 6))
plt.semilogy(snr_db, ber, 'bo-', label='Simulated BER (OTFS)')
plt.semilogy(snr_db, theoretical_ber, 'r--', label='Theoretical BER (BPSK)')
plt.xlabel('SNR (dB)')
plt.ylabel('Bit Error Rate (BER)')
plt.title('BER vs SNR for OTFS Modulation')
plt.grid(True, which="both", ls="--")
plt.ylim(1e-6, 1)  # Set BER limits
plt.legend()
plt.show()